# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'Big Mac'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
import string
listaR = []
palavrasSim = []
palavrasNao = []
#Limpa a planilha com palavras usáveis
planilhaTREI = pd.read_excel('Big Mac TREINAMENTO.xlsx',sep=',')
planilhaTEST = pd.read_excel('Big Mac TESTE.xlsx',sep=',')
for i in range(len(planilhaTREI.Treinamento)):
    linhaX = planilhaTREI.Treinamento[i].lower()
    linhaX = linhaX.split()
    for k in range(len(linhaX)):
        for punctuation in string.punctuation:
            linhaX[k] = linhaX[k].replace(punctuation, '')
        linhaX[k] = linhaX[k].replace('—', '')
        linhaX[k] = linhaX[k].replace('rt', '')
        if planilhaTREI.Relevância[i] == 'sim':
            palavrasSim.append(linhaX[k])
        elif planilhaTREI.Relevância[i] == 'não':
            palavrasNao.append(linhaX[k])
    while '' in linhaX:
        linhaX.remove('')
    planilhaTREI.Treinamento[i] = linhaX

#Remove '' da lista de palavras a serem contadas    
while '' in palavrasSim:
    palavrasSim.remove('')

while '' in palavrasNao:
    palavrasNao.remove('')
    
#Calcula quantas vezes aparece Sim ou Nao    
for i in planilhaTREI.Relevância:
    if i == 'sim':
        listaR.append(i)
    if i == 'não':
        listaR.append(i)
numeroSim = 0
numeroNao = 0
for j in listaR:
    if j == 'sim':
        numeroSim += 1
    if j == 'não':
        numeroNao += 1
        
#Conta cada palavra da lista
contandoSim = [[x,palavrasSim.count(x)] for x in set(palavrasSim)]
contandoNao = [[y,palavrasNao.count(y)] for y in set(palavrasNao)]


#Calcula quantas palavras existem no espaço amostral
wordsNumber = 0
numeroPalavrasSim = 0
numeroPalavrasNao = 0
for k in range(len(contandoSim)):
    wordsNumber = wordsNumber + contandoSim[k][1]
    numeroPalavrasSim = numeroPalavrasSim + contandoSim[k][1]
for k in range(len(contandoNao)):
    wordsNumber = wordsNumber + contandoNao[k][1]
    numeroPalavrasNao = numeroPalavrasNao + contandoNao[k][1]


#print(contandoSim)
#print('.')
#print(contandoNao)
print(numeroSim)
print(numeroNao)
print('% sim',numeroSim/300)
print('% nao',numeroNao/300)
print('Total de palavras', len(contandoSim)+len(contandoNao))
print('Total de palavras relevantes', len(contandoSim))
print('Total de palavras não relevantes', len(contandoNao))

print(listaR)
#planilhaTEST

#Limpando a nova planilha
for i in range(len(planilhaTEST.Teste)):
    linhaX = planilhaTEST.Teste[i].lower()
    linhaX = linhaX.split()
    for k in range(len(linhaX)):
        for punctuation in string.punctuation:
            linhaX[k] = linhaX[k].replace(punctuation, '')
        linhaX[k] = linhaX[k].replace('—', '')
        linhaX[k] = linhaX[k].replace('rt', '')
    while '' in linhaX:
        linhaX.remove('')
    planilhaTEST.Teste[i] = linhaX
    
#Calcular a probabilidade de uma frase ser relevante na planilha teste
probAllSIM = []
y = 1
for i in range(len(planilhaTEST.Teste)):
    probLinha = []
    for k in range(len(planilhaTEST.Teste[i])):
        probX = 0
        for j in range(len(contandoSim)):
            if contandoSim[j][0] == planilhaTEST.Teste[i][k]:
                probX = ((contandoSim[j][1]+1)/(len(contandoSim)+(len(contandoSim)+len(contandoNao))))
                break
        if probX > 0:
            probLinha.append(probX)
        elif probX == 0:
            probLinha.append(1/(len(contandoSim)+(len(contandoSim)+len(contandoNao))))
    y = 1
    for x in probLinha:
        y *= x
    probAllSIM.append(y)


#Calcular a probabilidade de uma frase não ser relevante na planilha teste
probAllNAO = []
y = 1
for i in range(len(planilhaTEST.Teste)):
    probLinha = []
    for k in range(len(planilhaTEST.Teste[i])):
        probX = 0
        for j in range(len(contandoNao)):
            if contandoNao[j][0] == planilhaTEST.Teste[i][k]:
                probX = ((contandoNao[j][1]+1)/(len(contandoNao)+(len(contandoSim)+len(contandoNao))))
                break
        if probX > 0:
            probLinha.append(probX)
        elif probX == 0:
            probLinha.append(1/(len(contandoNao)+(len(contandoSim)+len(contandoNao))))
    y = 1
    for x in probLinha:
        y *= x
    probAllNAO.append(y)

#print(probAllSIM)
#print('.')
#print(probAllNAO)
#print('Prob all sim',len(probAllSIM))
#print('Prob all nao',len(probAllNAO))

#Comapra a probabilidade da palavra ser relevante ou nao e cria uma planilha para isso
listaRelevanciaNova = []
for i in range(len(probAllSIM)):
    if probAllSIM[i]>probAllNAO[i]:
        listaRelevanciaNova.append('sim')
    elif probAllSIM[i]<probAllNAO[i]:
        listaRelevanciaNova.append('não')

print(listaRelevanciaNova)


208
92
% sim 0.6933333333333334
% nao 0.30666666666666664
Total de palavras 1156
Total de palavras relevantes 673
Total de palavras não relevantes 483
['sim', 'não', 'sim', 'sim', 'sim', 'não', 'sim', 'não', 'não', 'não', 'sim', 'não', 'não', 'não', 'sim', 'não', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'não', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'não', 'não', 'sim', 'sim', 'sim', 'não', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'não', 'não', 'sim', 'não', 'sim', 'sim', 'não', 'sim', 'sim', 'não', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'não', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'não', 'não', 'não', 'sim', 'sim', 'não', 'não', 'sim', 'sim', 'sim', 'não', 'sim', 'sim', 'sim', 'não', 'sim', 'sim', 'não', 'não', 'sim', 'sim', 'não', 'sim', 'sim', 'sim', 'sim', 'sim', 'sim', 'não', 'sim', 'sim', 'sim', 'sim', 'sim', 'não', 'sim', 'sim', 'sim', 'sim', 'sim', 'não', 'não', 'sim', 'sim', 'sim', 'não', 'sim', 'sim', 'sim', 'sim', '

In [9]:
planilhaTEST['Relevância Nova calculada'] = listaRelevanciaNova
ct = pd.crosstab(planilhaTEST['Relevancia'],planilhaTEST['Relevância Nova calculada'], margins=True)
#planilhaTREI
ct

Relevância Nova calculada,não,sim,All
Relevancia,,,
não,29,37,66
sim,1,133,134
All,30,170,200


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [10]:
print("Relevantes - Classificados Incorretamente", (37/66)*100)
print("Relevantes - Classificados Corretamente", (133/134)*100)
print("Não relevantes - Classificados Corretamente", (29/66)*100)
print("Não relevantes - Classificados Incorretamente", (1/134)*100)
print("% de acertos", ((29+133)/200)*100)
print("% de erros", (38/200)*100)

Relevantes - Classificados Incorretamente 56.060606060606055
Relevantes - Classificados Corretamente 99.25373134328358
Não relevantes - Classificados Corretamente 43.93939393939394
Não relevantes - Classificados Incorretamente 0.7462686567164178
% de acertos 81.0
% de erros 19.0


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Ao observar-se a Crosstab comparativa dos resultados (classificação feita pelos alunos X cálculo pelo dispositivo Naive-Bayes), nota-se uma ótima precisão ao classificar-se as frases que julgamos relevantes (99% de acertos). Contudo, houve um erro expressivamente maior quando analisadas as frases não relevantes: parte dessas frases foram postas como relevantes, sendo que deviam ser classificadas como não-relevantes (56% foram classificadas erroneamente).  

Dado que nosso classificador é bem simples (não possuindo processos lógicos de classificação complexos, os quais englobariam uma maior gama de tipos de construção de frases), expressões opostas ao que se realmente pretende dizer (sarcásticas), por exemplo, seriam detectadas como relevantes ao invés do sentido real, irrelevantes e vice-versa. Frases de dupla negação resultariam no mesmo problema: a classificação pelo naive-bayes não chega a um alto nível de interpretação de texto, ou seja, não detectaria que a mensagem na verdade tem um sentido oposto ao qual está escrito. 

Nosso projeto já demonstrou eficiência na detecção de frases relevantes. Porém, essa detecção ainda está parcialmente desrregulada, classificando parte das frases não relevantes também como relevantes. De qualquer forma, a CrossTab demonstra que os erros totais foram baixos, na casa dos 20%. Algumas iterações na lógica de classificação nos traria resultados ainda mais precisos.





Alimentar a base de treinamento com o próprio classificador é inútil, pois, se o próprio classificador dá os parâmetros de comparação para o que ele faria nas próximas bases de dados, não haveria diferença qualquer nos critérios de classificação, ou seja, não haveria parâmetros "reais", apenas parâmetros criados pelo próprio classificador. 

O classificador Naive-Bayes possui larga aplicação no que se refere a categorização de textos. Então, não está preso a detectar sentimentos, como neste projeto: ele pode ser usado, por exemplo, para detectar textos de spam em relação a e-mails legítmos. 
Indo mais além, encontra-se utilidade no navegador também na Medicina: o classificador consegue ajudar o profissional a tomar decisões e de forma mais rápida em diagnósticos, tudo isso por meio da probabilidade. 
Fonte: http://scialert.net/fulltext/?doi=itj.2012.1166.1174&org=11